# 思路
- 分析用户特征与流失的关系
- 从整体情况看，流失用户的普遍特征是什么
- 尝试找到合适的模型预测流失用户
- 针对性给出增加用户粘性、预防流失的建议

## 理解数据
- 数据集字段情况

## 数据清洗
- 完整性 ：单条数据是否存在空值，统计字段是否完善
- 全面性 ：观察某一列的全部数值，通过常识来判断该列是否有问题，比如：数据定义、单位标识、数据本身
- 合法性 ：数据的类型、内容、大小的合法性
- 唯一性 ：数据是否存在重复记录，因为数据通常来自不同渠道的汇总，重复的情况是常见的。行数据、列数据都需要是唯一的

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
customer = pd.read_csv("./dataset/WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [3]:
# 数据机大小
customer.shape

(7043, 21)

In [4]:
# 设置查看列，不省略
pd.set_option("display.max_columns",None)

In [5]:
customer.head(10)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
5,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes
6,1452-KIOVK,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.4,No
7,6713-OKOMC,Female,0,No,No,10,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,No,Mailed check,29.75,301.9,No
8,7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes
9,6388-TABGU,Male,0,No,Yes,62,Yes,No,DSL,Yes,Yes,No,No,No,No,One year,No,Bank transfer (automatic),56.15,3487.95,No


In [6]:
# 查看空值
pd.isnull(customer).sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [7]:
# 查看数据类型
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [8]:
# 查看每一列数据值，检查每个字段的数据类型、字段内容、字段数量
for x in customer.columns:
    test = customer.loc[:,x].value_counts()
    print('{0} 的行数是： {1}'.format(x,test.sum()))
    print('{0} 的数据类型是：{1}'.format(x,customer[x].dtypes))
    print('{0} 的内容是：{1}'.format(x,test))

customerID 的行数是： 7043
customerID 的数据类型是：object
customerID 的内容是：0907-HQNTS    1
4817-QRJSX    1
9224-VTYID    1
4913-EHYUI    1
3404-JNXAX    1
             ..
7242-QZLXF    1
6366-ZGQGL    1
9424-CMPOG    1
7314-OXENN    1
6344-SFJVH    1
Name: customerID, Length: 7043, dtype: int64
gender 的行数是： 7043
gender 的数据类型是：object
gender 的内容是：Male      3555
Female    3488
Name: gender, dtype: int64
SeniorCitizen 的行数是： 7043
SeniorCitizen 的数据类型是：int64
SeniorCitizen 的内容是：0    5901
1    1142
Name: SeniorCitizen, dtype: int64
Partner 的行数是： 7043
Partner 的数据类型是：object
Partner 的内容是：No     3641
Yes    3402
Name: Partner, dtype: int64
Dependents 的行数是： 7043
Dependents 的数据类型是：object
Dependents 的内容是：No     4933
Yes    2110
Name: Dependents, dtype: int64
tenure 的行数是： 7043
tenure 的数据类型是：int64
tenure 的内容是：1     613
72    362
2     238
3     200
4     176
     ... 
28     57
39     56
44     51
36     50
0      11
Name: tenure, Length: 73, dtype: int64
PhoneService 的行数是： 7043
PhoneService 的数据类型是：object
PhoneServ

In [9]:
# 强制转换总消费额为数字，转换为浮点型数据,不可转换的变为NaN
customer['TotalCharges'] =pd.to_numeric(customer['TotalCharges'],errors='coerce') 

In [10]:
test2=customer['TotalCharges'].value_counts().sort_index()
print(test2.sum())

7032


In [11]:
customer.tenure[customer['TotalCharges'].isnull().values == True]

488     0
753     0
936     0
1082    0
1340    0
3331    0
3826    0
4380    0
5218    0
6670    0
6754    0
Name: tenure, dtype: int64

In [12]:
customer.isnull().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [13]:
customer[customer['TotalCharges'].isnull().values==True]

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,4472-LVYGI,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,NaN,No
753,3115-CZMZD,Male,0,No,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,NaN,No
936,5709-LVOEQ,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,NaN,No
1082,4367-NUYAO,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,NaN,No
1340,1371-DWPAZ,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,NaN,No
3331,7644-OMVMY,Male,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,NaN,No
3826,3213-VVOLG,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,NaN,No
4380,2520-SGTTA,Female,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,NaN,No
5218,2923-ARZLG,Male,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,NaN,No
6670,4075-WKNIU,Female,0,Yes,Yes,0,Yes,Yes,DSL,No,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,NaN,No


In [14]:
# 将总消费额填充为 月消费额
customer['TotalCharges'].replace(to_replace=np.nan,value=customer['MonthlyCharges'],inplace=True)
customer[customer['tenure']==0]

ValueError: Series.replace cannot use dict-value and non-None to_replace

In [ ]:
# 将入网时长 tenure 从0 改为1
customer['tenure'].replace(to_replace=0, value=1,inplace=True)
customer.isnull().sum()

In [ ]:
customer['TotalCharges'].dtypes

In [ ]:
# 数据的描述统计信息
customer.describe()

## 可视化分析

In [ ]:
plt.pie(customer['Churn'].value_counts(),labels=customer['Churn'].value_counts().index,autopct='%1.2f%%',explode=(0.1,0))

In [ ]:
churn = customer['Churn'].value_counts().to_frame()
x = churn.index
y = churn['Churn']
plt.bar(x,y,width=0.5,color='c')

In [ ]:
g =(customer.groupby('gender')['Churn'].value_counts() / len(customer)).to_frame().rename(columns={"Churn":"percentage"}).reset_index()
g

In [ ]:
# 用户属性分析
ax =sns.barplot(x= 'gender',y='percentage',hue='Churn',data=g )
plt.title("Churn yes/no  &&&  nan nv qubie  ")
ax.set_yticklabels(['{:,.0%}'.format(x) for x in ax.get_yticks()])

In [ ]:
g1 = (customer.groupby('SeniorCitizen')['Churn'].value_counts()/ len(customer)).to_frame().rename(columns={"Churn":"percentage"}).reset_index()
g1

In [ ]:
ax1 = sns.barplot(x='SeniorCitizen',y='percentage',hue='Churn',data=g1)
plt.title("Churn yes/no  &&&  shi fou lai nian ren  ")
ax1.set_yticklabels(['{:,.0%}'.format(x) for x in ax1.get_yticks()])
plt.show()

In [ ]:
customer['churn_rate'] = customer['Churn'].replace("No",0).replace("Yes",1)
customer['churn_rate']
g2 = sns.FacetGrid(customer,col="SeniorCitizen",height=4,aspect=.9)
ax2 = g2.map(sns.barplot,"gender","churn_rate",palette="Blues_d",order=['Female','Male'])

#### 用户流失与性别基本无关； 年老用户流失明显 高于年轻用户

In [ ]:
fig,axis = plt.subplots(1,2,figsize=(12,4))
axis[0].set_title("Has Partner")
axis[1].set_title("Has Dependents")
axis_y = "percentage of customers"

gp_partner = (customer.groupby("Partner")["Churn"].value_counts()/len(customer)).to_frame()
gp_partner.rename(columns={"Churn":axis_y},inplace=True)
gp_partner.reset_index(inplace=True)
ax1 = sns.barplot(x='Partner',y="percentage of customers",hue='Churn',data=gp_partner,ax= axis[0])
# ax1.set_xlabel("Partner")


gp_dep = (customer.groupby("Dependents")["Churn"].value_counts()/len(customer)).to_frame()
gp_dep.rename(columns={"Churn":axis_y}, inplace=True)
gp_dep.reset_index(inplace=True)
ax2 = sns.barplot(x='Dependents',y=axis_y,hue='Churn',data=gp_dep ,ax =axis[1])


- 有伴侣的用户流失占比低于 无伴侣用户
- 有家属的用户较少
- 有家属的用户流失占比 低于无家属用户

In [ ]:
# Kernel density estimation核密度估计
def kdeplot(feature,xlabel):
    plt.figure(figsize=(9,4))
    plt.title("KDE for {0}".format(feature))
    ax0 = sns.kdeplot(customer[customer['Churn'] == 'No'][feature].dropna(), label='Churn:No',shade='True' )
    ax1 = sns.kdeplot(customer[customer['Churn'] == 'Yes'][feature].dropna(), label='Churn:Yes',shade='True')
    
    plt.legend()
kdeplot('tenure','tenure')
plt.show()

- 在网时长越久，流失率越低
- 在网时长22左右的时候，流失率 低于 在网率

In [ ]:
def barplot_percentages(feature,orient,axis_name="percentage of customers"):
    ratios = pd.DataFrame()
    g = (customer.groupby(feature)['Churn'].value_counts() / len(customer)).to_frame()
    g.rename(columns={"Churn":axis_name},inplace = True)
    g.reset_index(inplace=True)
    
    if orient == 'v':
        ax = sns.barplot(x= feature, y=axis_name ,hue='Churn' ,data=g , orient = orient)
        ax.set_yticklabels(['{:,.0%}'.format(i) for i in ax.get_yticks()])
    else:
        ax = sns.barplot(x=axis_name, y =feature , hue='Churn' ,data=g , orient = orient)
        ax.set_xticklabels(['{:,.0%}'.format(k) for k in ax.get_xticks()])
    plt.title('Churn(yes/no) Ratio as {0}'.format(feature))
    plt.show()

### 服务属性分析

In [ ]:
plt.figure(figsize=(9,4.5))
barplot_percentages("MultipleLines",orient='h')

In [ ]:
plt.figure(figsize=(9,4.5))
barplot_percentages("InternetService",orient="h")

In [ ]:
cols = ["PhoneService","MultipleLines","OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies"]
d_col=customer[customer["InternetService"] != "No"][cols]
df1 = pd.melt(d_col)
df1.rename(columns={'value':'has service'},inplace=True)
plt.figure(figsize=(20,8))
ax = sns.countplot(data= df1, x= "variable" , hue = 'has service')
ax.set(xlabel='Internet Additional service' , ylabel='Num of customers')
plt.title('Num of customers as Internet Additional Service')
plt.show()

In [ ]:
plt.figure(figsize=(20,8))
df1 = customer[(customer['InternetService'] != "No") & (customer['Churn']== "Yes")]
df1 = pd.melt(df1[cols])
df1.rename(columns={'value':'has service'},inplace =True)
ax = sns.countplot(data=df1,x='variable',hue='has service',hue_order =['No','Yes'])
ax.set(xlabel='Internet Additional service',ylabel = 'Churn Num')
plt.title('Num of Churn Customers as Internet Additional Service')
plt.show()

- 电话服务整体对用户流失影响较小
- 绑定了安全、备份、保护、技术支持服务的流失率较低
- 附加流媒体点时、电影服务的流失率占比较高

### 合同属性分析

In [ ]:
plt.figure(figsize=(9,4.5))
barplot_percentages("PaymentMethod",orient='h')

In [ ]:
g = sns.FacetGrid(customer,col="PaperlessBilling", height=6,aspect =.9)
ax = g.map(sns.barplot,"Contract","churn_rate",palette="Blues_d",order= ['Month-to-month', 'One year', 'Two year'])
plt.show()

In [ ]:
kdeplot('MonthlyCharges','MonthlyCharges')
kdeplot('TotalCharges','TotalCharges')
plt.show()

- 月消费金额70-110的流失率更高
- 支票支付的用户流失率更高
- 长期来看，用户总消费越高，流失率越低，符合一般经验

### 用户流失预测

- 对数据集进行进一步清洗  和特征提取
- 对特征选取数据进行降维
- 采用机器学习模型应用于测试数据集
- 对构建的分类模型准确性进行分析

##### 数据清洗

- axis 使用0值表示沿着每一列或行标签\索引值向下执行方法
- axis 使用1值表示沿着每一行或者列标签模向执行对应的方法

观察数据类型， 多数为离散特征
- 对于连续特征，采用标准化方式处理
- 对于离散特征，特征之间没有大小关系，采用 one-hot编码
- 特征之间有大小关联，采用数值映射

In [41]:
customerId = customer['customerID']
customerId
customer.drop(['customerID'] ,axis=1,inplace=True)

0       7590-VHVEG
1       5575-GNVDE
2       3668-QPYBK
3       7795-CFOCW
4       9237-HQITU
5       9305-CDSKC
6       1452-KIOVK
7       6713-OKOMC
8       7892-POOKP
9       6388-TABGU
10      9763-GRSKD
11      7469-LKBCI
12      8091-TTVAX
13      0280-XJGEX
14      5129-JLPIS
15      3655-SNQYZ
16      8191-XWSZG
17      9959-WOFKT
18      4190-MFLUW
19      4183-MYFRB
20      8779-QRDMV
21      1680-VDCWW
22      1066-JKSGK
23      3638-WEABW
24      6322-HRPFA
25      6865-JZNKO
26      6467-CHFZW
27      8665-UTDHZ
28      5248-YGIJN
29      8773-HHUOZ
           ...    
7013    1685-BQULA
7014    9053-EJUNL
7015    0666-UXTJO
7016    1471-GIQKQ
7017    4807-IZYOZ
7018    1122-JWTJW
7019    9710-NJERN
7020    9837-FWLCH
7021    1699-HPSBG
7022    7203-OYKCT
7023    1035-IPQPU
7024    7398-LXGYX
7025    2823-LKABH
7026    8775-CEBBJ
7027    0550-DCXLH
7028    9281-CEDRU
7029    2235-DWLJU
7030    0871-OPBXW
7031    3605-JISKB
7032    6894-LFHLY
7033    9767-FFLEM
7034    0639